    Если спросить питонистов, какую библиотеку использовать для визуализации данных, то большинство, несомненно, ответят: matplotlib или Seaborn. Ответят, правда, без особого удовольствия. Многое в этих библиотеках не так уж очевидно и требует не раз обратиться к StackOverflow. Касается это и таких распространенных ситуаций, как создание дополнительных осей или отображение на них дат.
	Нужно понимать, что matplotlib создавался задолго до бурного развития Data Science, и в большей мере ориентировался на отображение массивов NumPy и параметрических функций SciPy. В то же время в Data Science распространен обобщающий тип объектов – датасеты, крупные таблицы с данными. Но существует альтернативная библиотека, которая позволяет создавать красивые, интерактивные, экспортируемые графики с помощью всего нескольких строк кода plotly. Её функционал намного шире чем у предшественников, и она дает больше возможностей.
	Plotly — библиотека для визуализации данных, состоящая из нескольких частей:
    • Front-End на JS
    • Back-End на Python (за основу взята библиотека Seaborn)
    • Back-End на R